### Packages

In [1]:
#Importing Relevant Libraries
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading the data

In [2]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

[*********************100%***********************]  4 of 4 completed


In [3]:
df_comp = raw_data.copy()

In [4]:
df_comp.head(10)

^FTSE                                                \
                   Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0   
1994-01-14  3370.899902  3405.600098  3369.399902  3400.600098    0.0   
1994-01-17  3405.399902  3421.100098  3397.399902  3407.800049    0.0   
1994-01-18  3416.500000  3439.600098  3410.000000  3437.000000    0.0   
1994-01-19  3451.199951  3484.500000  3451.199951  3475.100098    0.0   
1994-01-20  3463.199951  3476.000000  3458.899902  3470.000000    0.0   

                   ^N225                                                   \
                    Open          High           Low         Close Volume   
Date                                                                        
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766    0.0   
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453    0.0   
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000    0.0   
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859    0.0   
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766    0.0   
1994-01-14  18623.460938  19043.800781  18473.349609  18973.699219    0.0   
1994-01-17  18911.890625  18911.890625  18646.060547  18725.369141    0.0   
1994-01-18  18703.669922  18810.279297  18514.550781  18514.550781    0.0   
1994-01-19  18516.050781  19118.810547  18515.650391  19039.400391    0.0   
1994-01-20  19072.509766  19341.730469  19025.699219  19183.919922    0.0   

                 ^GSPC                                                   \
                  Open        High         Low       Close       Volume   
Date                                                                      
1994-01-07  467.089996  470.260010  467.029999  469.899994  324920000.0   
1994-01-10  469.899994  475.269989  469.549988  475.269989  319490000.0   
1994-01-11  475.269989  475.279999  473.269989  474.130005  305490000.0   
1994-01-12  474.130005  475.059998  472.140015  474.170013  310690000.0   
1994-01-13  474.170013  474.170013  471.799988  472.470001  277970000.0   
1994-01-14  472.500000  475.320007  472.500000  474.910004  304920000.0   
1994-01-17  474.910004  474.910004  472.839996  473.299988  233980000.0   
1994-01-18  473.299988  475.190002  473.290009  474.250000  308840000.0   
1994-01-19  474.250000  474.700012  472.209991  474.299988  311370000.0   
1994-01-20  474.299988  475.000000  473.420013  474.980011  310450000.0   

                 ^GDAXI                                                
                   Open         High          Low        Close Volume  
Date                                                                   
1994-01-07  2218.959961  2227.639893  2201.820068  2224.949951    0.0  
1994-01-10  2231.840088  2238.010010  2222.000000  2225.000000    0.0  
1994-01-11  2225.429932  2235.610107  2225.179932  2228.100098    0.0  
1994-01-12  2227.120117  2227.790039  2182.060059  2182.060059    0.0  
1994-01-13  2171.500000  2183.709961  2134.100098  2142.370117    0.0  
1994-01-14  2151.199951  2153.350098  2128.449951  2151.050049    0.0  
1994-01-17  2153.159912  2155.729980  2113.030029  2115.560059    0.0  
1994-01-18  2108.330078  2131.250000  2105.239990  2130.350098    0.0  
1994-01-19  2133.320068  2138.979980  2128.830078  2132.520020    0.0  
1994-01-20  2133.290039  2136.419922  2093.800049  2098.360107    0.0

In [5]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [6]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

### Creating Returns

In [7]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

### Splitting the Data

In [8]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a Model

In [9]:
from pmdarima import auto_arima

In [10]:
model_auto = auto_arima(df_comp.ret_ftse[1:])

In [11]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True)

In [12]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 6275
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -9555.091
Date:                Thu, 01 Apr 2021   AIC                          19132.182
Time:                        16:39:55   BIC                          19206.369
Sample:                             0   HQIC                         19157.887
                               - 6275                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0316      0.020      1.556      0.120      -0.008       0.071
ar.L1          0.0134      0.076      0.176      0.861      -0.136       0.163
ar.L2         -0.6077      0.075     -8.142      0.000      -0.754      -0.461
ar.L3         -0.1858      0.068     -2.732      0.006      -0.319      -0.053
ar.L4          0.2443      0.072      3.393      0.001       0.103       0.385
ma.L1         -0.0372      0.076     -0.487      0.626      -0.187       0.112
ma.L2          0.5600      0.075      7.420      0.000       0.412       0.708
ma.L3          0.0973      0.066      1.465      0.143      -0.033       0.228
ma.L4         -0.2471      0.071     -3.463      0.001      -0.387      -0.107
ma.L5         -0.0935      0.009    -10.980      0.000      -0.110      -0.077
sigma2         1.2292      0.012    104.078      0.000       1.206       1.252
===================================================================================
Ljung-Box (L1) (Q):                   0.05   Jarque-Bera (JB):              8643.43
Prob(Q):                              0.83   Prob(JB):                         0.00
Heteroskedasticity (H):               0.87   Skew:                            -0.19
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.74
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Important Arguments

In [13]:
model_auto = auto_arima(df_comp.ret_ftse[1:], exogenous=df_comp[['ret_spx','ret_dax','ret_nikkei']][1:], m = 5, max_order = None, 
                        max_p = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2, maxiter = 50, alpha = 0.05, n_jobs = -1,
                       trend = "ct", information_criterion = "oob", out_of_sample_size = int(len(df_comp) * 0.2))
# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# return_valid_fits -> whether or not the method should validate the results 
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple_size -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [14]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 6275
Model:             SARIMAX(3, 0, 1)x(1, 0, [1, 2], 5)   Log Likelihood               -6347.774
Date:                                Thu, 01 Apr 2021   AIC                          12721.548
Time:                                        18:42:18   BIC                          12809.225
Sample:                                             0   HQIC                         12751.927
                                               - 6275                                         
Covariance Type:                                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0067      0.017     -0.401      0.689      -0.039       0.026
drift      -7.364e-07   5.08e-06     -0.145      0.885   -1.07e-05    9.21e-06
x1             0.0854      0.006     13.757      0.000       0.073       0.098
x2             0.5634      0.005    103.429      0.000       0.553       0.574
x3             0.0739      0.005     15.675      0.000       0.065       0.083
ar.L1         -0.0583      0.127     -0.457      0.647      -0.308       0.191
ar.L2         -0.0166      0.018     -0.937      0.349      -0.051       0.018
ar.L3         -0.0686      0.010     -6.706      0.000      -0.089      -0.049
ma.L1         -0.0613      0.128     -0.479      0.632      -0.312       0.189
ar.S.L5       -0.0113      0.183     -0.062      0.951      -0.370       0.348
ma.S.L5       -0.0216      0.183     -0.119      0.906      -0.379       0.336
ma.S.L10      -0.0550      0.012     -4.540      0.000      -0.079      -0.031
sigma2         0.4870      0.005     89.705      0.000       0.476       0.498
===================================================================================
Ljung-Box (L1) (Q):                   2.09   Jarque-Bera (JB):             12878.04
Prob(Q):                              0.15   Prob(JB):                         0.00
Heteroskedasticity (H):               0.55   Skew:                             0.22
Prob(H) (two-sided):                  0.00   Kurtosis:                        10.00
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
model_auto_seasonal = auto_arima(df_comp.ret_ftse[1:], exogenous=df_comp[['ret_spx','ret_dax','ret_nikkei']][1:], m = 5,
                                 seasonal=True, max_order = None, max_p = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2, 
                                 maxiter = 50, alpha = 0.05, n_jobs = -1,trend = "ct", information_criterion = "oob", 
                                 out_of_sample_size = int(len(df_comp) * 0.2))